## Text Named Entity Recognition Evaluation

This sample shows how use the evaluate a group of models against a given set of metrics for the `text-named-entity-recognition` task. 

### Evaluation dataset
The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2 tagging scheme, whereas the original dataset uses IOB1. Reference [CoNLL-2003](https://huggingface.co/datasets/conll2003).

### Model
The goal of evaluating models is to compare their performance on a variety of metrics. `text-named-entity-recognition` is generic task type that can be used for scenarios to recognise named entities such as persons, locations, organizations, etc. As such, the models you pick to compare must be finetuned for same scenario. Given that we have the CoNLL-2003 dataset, we would like to look for models finetuned for this specific scenario. We will review `jean-baptiste-camembert-ner` in this sample, which is available in the `azureml` system registry.

If you'd like to evaluate models that are not in the system registry, you can import those models to your workspace or organization registry and then evaluate them using the approach outlined in this sample.

### Outline
* Setup pre-requisites such as compute.
* Pick the models to evaluate.
* Pick and explore evaluate data.
* Configure the evaluation jobs.
* Run the evaluation jobs.
* Review the evaluation metrics. 

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry
* Set an optional experiment name
* Check or create compute. A single GPU node can have multiple GPU cards. For example, in one node of `STANDARD_NC4AS_T4_V3` there are 4 NVIDIA V100 GPUs while in `STANDARD_NC4AS_T4_V3`, there are 2 NVIDIA V100 GPUs. Refer to the [docs](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu) for this information. The number of GPU cards per node is set in the param `gpus_per_node` below. Setting this value correctly will ensure utilization of all GPUs in the node. The recommended GPU compute SKUs can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ncv3-series) and [here](https://learn.microsoft.com/en-us/azure/virtual-machines/ndv2-series).

Install dependencies by running below cell. This is not an optional step if running in a new environment.

In [ ]:
import pandas as pd

%pip install --upgrade azure-ai-ml
%pip install --upgrade azure-identity
%pip install --upgrade datasets==2.9.0

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    workspace = workspace_ml_client.workspace_name
    resource_group = workspace_ml_client.resource_group_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    workspace_ml_client = MLClient(
        credential, subscription_id, resource_group, workspace
    )

# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry = "azureml"

registry_ml_client = MLClient(
    credential, subscription_id, resource_group, registry_name=registry
)
registry_ml_client

### Creating Compute

There are two ways to submit a job - through a compute or a serverless job.

##### Serverless Job:

In a serverless job, there is no need to create a compute explicitly.
Simply pass the desired instance type value to the `instance_type` parameter while creating a pipeline job.
This allows for quick and convenient job submission without the need for managing a compute cluster.

##### Compute Job:

To submit a job through a compute, you need to create a compute cluster beforehand.
The following code below demonstrates how to create a gpu compute cluster.
After creating the compute cluster, pass the name of the compute cluster to the `compute_name` parameter while submitting the pipeline job. This ensures that the job runs on the specified compute cluster, allowing for more control and customization.

In [ ]:
# If you already have a gpu cluster, mention it here. Else will create a new one with the name 'gpu-cluster-big'
compute_cluster = "gpu-cluster-big"
try:
    compute = workspace_ml_client.compute.get(compute_cluster)
    print(f"GPU compute '{compute_cluster}' found.")
except Exception as ex:
    print(f"GPU compute '{compute_cluster}' not found. Creating new one.")
    compute = AmlCompute(
        name=compute_cluster,
        size="STANDARD_NC4AS_T4_V3",
        max_instances=2,  # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

Below snippet will allow us to query number of GPU's present on the compute. We can use it to set `gpu_per_node` to ensure utilization of all GPUs in the node.

In [ ]:
# This is the number of GPUs in a single node of the selected 'vm_size' compute.
# Setting this to less than the number of GPUs will result in underutilized GPUs, taking longer to train.
# Setting this to more than the number of GPUs will result in an error.
gpus_per_node = 1  # default value
gpu_count_found = False
ws_computes = workspace_ml_client.compute.list_sizes()
for ws_compute in ws_computes:
    if ws_compute.name.lower() == compute.size.lower():
        gpus_per_node = ws_compute.gpus
        print(f"Number of GPUs in compute {ws_compute.name} are {ws_compute.gpus}")
# if gpu_count_found not found, then print an error
if gpus_per_node > 0:
    gpu_count_found = True
else:
    gpu_count_found = False
    print(f"No GPUs found in compute. Number of GPUs in compute {compute.size} 0.")

### 2. Pick the models to evaluate

Verify that the models selected for evaluation are available in system registry

In [ ]:
# need to specify model versions until the bug to support fetching the latest version using latest label is fixed
model_details = [
    {"name": "Jean-Baptiste-camembert-ner"},
]

In [ ]:
models = []

for model in model_details:
    reg_model = list(registry_ml_client.models.list(name=model["name"]))[0]
    print(reg_model.id)
    models.append({**model, "version": reg_model.version})

In [ ]:
models

### 3. Pick the test dataset for evaluation
The next few cells show basic data preparation:
* Visualize some data rows
* We want this sample to run quickly, so we use a smaller dataset containing 10% of the original.
* To use the entire dataset, uncomment the cells below and run.

In [ ]:
from datasets import load_dataset

hf_test_data = load_dataset("conll2003", split="test", streaming=True)

test_data_df = pd.DataFrame(hf_test_data.take(1000))

In [ ]:
# Picked from https://huggingface.co/datasets/conll2003
label_dict = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-MISC": 7,
    "I-MISC": 8,
}

In [ ]:
label_reverse_dict = {value: key for key, value in label_dict.items()}
label_reverse_dict

In [ ]:
test_data_df["input_string"] = test_data_df["tokens"].apply(lambda x: " ".join(x))
test_data_df["ner_tags_str"] = test_data_df["ner_tags"].apply(
    lambda x: str([label_reverse_dict[tag] for tag in x])
)

In [ ]:
test_data_df.head()

In [ ]:
test_data = "./small-test.jsonl"

In [ ]:
test_data_df.to_json(test_data, lines=True, orient="records")

In [ ]:
pd.read_json(test_data, lines=True).head()

### 4. Submit the evaluation jobs using the model and data as inputs

Create the job that uses the `model_evaluation_pipeline` component. We will submit one job per model.

Note that the metrics that the evaluation jobs need to calculate are specified in the [eval_config.json](./eval_config.json) file. We calculate `accuracy`, `f1_score_macro` and `f1_score_micro` in this sample.

All supported evaluation configurations for `text-named-entity-recognition` can be found in [README](./README.md).

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(
    name="model_evaluation_pipeline", label="latest"
)


# define the pipeline job
@pipeline()
def evaluation_pipeline(mlflow_model):
    evaluation_job = pipeline_component_func(
        # specify the foundation model available in the azureml system registry or a model from the workspace
        # mlflow_model = Input(type=AssetTypes.MLFLOW_MODEL, path=f"{mlflow_model_path}"),
        mlflow_model=mlflow_model,
        # test data
        test_data=Input(type=AssetTypes.URI_FILE, path=test_data),
        # The following parameters map to the dataset fields
        input_column_names="input_string",
        label_column_name="ner_tags_str",
        # compute settings
        compute_name=compute_cluster,
        # specify the instance type for serverless job
        # instance_type= "STANDARD_NC24",
        # Evaluation settings
        task="text-named-entity-recognition",
        # config file containing the details of evaluation metrics to calculate
        evaluation_config=Input(type=AssetTypes.URI_FILE, path="./eval-config.json"),
        # config cluster/device job is running on
        # set device to GPU/CPU on basis if GPU count was found
        device="auto",
    )
    return {"evaluation_result": evaluation_job.outputs.evaluation_result}

Submit the jobs, passing the model as a parameter to the pipeline created in the above step.

In [ ]:
# submit the pipeline job for each model that we want to evaluate
# you could consider submitting the pipeline jobs in parallel, provided your cluster has multiple nodes
pipeline_jobs = []

experiment_name = "text-named-entity-recognition-evaluation"

for model in models:
    model_object = registry_ml_client.models.get(
        model["name"], version=model["version"]
    )
    pipeline_object = evaluation_pipeline(
        mlflow_model=Input(type=AssetTypes.MLFLOW_MODEL, path=f"{model_object.id}"),
    )
    # don't reuse cached results from previous jobs
    pipeline_object.settings.force_rerun = True
    pipeline_object.settings.default_compute = compute_cluster

    # set continue on step failure to False
    pipeline_object.settings.continue_on_step_failure = False

    pipeline_object.display_name = f"eval-{model['name']}-{timestamp}"
    pipeline_job = workspace_ml_client.jobs.create_or_update(
        pipeline_object, experiment_name=experiment_name
    )
    # add model['name'] and pipeline_job.name as key value pairs to a dictionary
    pipeline_jobs.append({"model_name": model["name"], "job_name": pipeline_job.name})
    # wait for the pipeline job to complete
    workspace_ml_client.jobs.stream(pipeline_job.name)

### 5. Review evaluation metrics
Viewing the job in AzureML studio is the best way to analyze logs, metrics and outputs of jobs. You can create custom charts and compare metics across different jobs. See https://learn.microsoft.com/en-us/azure/machine-learning/how-to-log-view-metrics?tabs=interactive#view-jobsruns-information-in-the-studio to learn more.

![Model evaluation dashboard in AzureML studio](./text-named-entity-recognition-eval-dashboard.png)

However, we may need to access and review metrics programmatically for which we will use MLflow, which is the recommended client for logging and querying metrics.

In [ ]:
import mlflow, json

mlflow_tracking_uri = workspace_ml_client.workspaces.get(
    workspace_ml_client.workspace_name
).mlflow_tracking_uri
mlflow.set_tracking_uri(mlflow_tracking_uri)

metrics_df = pd.DataFrame()
for job in pipeline_jobs:
    # concat 'tags.mlflow.rootRunId=' and pipeline_job.name in single quotes as filter variable
    filter = "tags.mlflow.rootRunId='" + job["job_name"] + "'"
    runs = mlflow.search_runs(
        experiment_names=[experiment_name], filter_string=filter, output_format="list"
    )
    # get the compute_metrics runs.
    # using a hacky way till 'Bug 2320997: not able to show eval metrics in FT notebooks - mlflow client now showing display names' is fixed
    for run in runs:
        # else, check if run.data.metrics.accuracy exists
        if "accuracy" in run.data.metrics:
            # get the metrics from the mlflow run
            run_metric = run.data.metrics
            # add the model name to the run_metric dictionary
            run_metric["model_name"] = job["model_name"]
            # convert the run_metric dictionary to a pandas dataframe
            temp_df = pd.DataFrame(run_metric, index=[0])
            # concat the temp_df to the metrics_df
            metrics_df = pd.concat([metrics_df, temp_df], ignore_index=True)

# move the model_name columns to the first column
cols = metrics_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
metrics_df = metrics_df[cols]
metrics_df.head()